# Your First TrackStar Program

This tutorial presents a simple TrackStar program to familiarize new users with its usage.
We'll construct a sample of mock data from a known input model, storing the data as a ``trackstar.sample`` object and the model predictions as a ``trackstar.track`` object.
We'll then compute the likelihood function with a few different settings for the same known input model.

We'll start by simply importing TrackStar and NumPy.

In [1]:
import trackstar
import numpy as np

/Users/jamesjohnson/Work/Research/lib/public-dev/TrackStar/trackstar/version.py:91: VersionWarning: Using a development version of TrackStar. Be advised that the features in development may exhibit buggy behavior and/or numerical artifacts. We encourage consulting with TrackStar developers.
  if self.dev is not None: warnings.warn("""\
/Users/jamesjohnson/Work/Research/lib/public-dev/TrackStar/trackstar/version.py:95: VersionWarning: Using an un-released version of TrackStar.
  if not self.isreleased: warnings.warn("""\


## A Mock Data Sample

Let's set up our mock sample.
For illustrative purposes, we'll keep it simple and straight-forward.

We'll take $N = 100$ points along the line $x = y = z$ in 3-dimensional space, following an intrinsic distribution that follows a Gaussian centered on zero (i.e. $\langle x \rangle = \langle y \rangle = \langle z \rangle = 0$) with a standard deviation of $\sigma = 1$.
To demonstrate TrackStar's user-friendliness in fitting non-uniform samples, we'll let only *half* of our data vectors have measurements of $z$.
Such instances may arise, e.g., within astrophysics, when not every star has a reliable age measurement.

We'll let $x$ and $y$ have measurement uncertainties of $\sigma_x = \sigma_y = 0.1$, while each measurement of $z$ will have a more substantial uncertainty of $\sigma_z = 0.3$.
In this instance, $z$ is a stand-in for some quantity that may be challenging to measure.
In real samples, there may be only a handful of coarse measurements, but the information is nonetheless useful or valuable.

Before bringing in TrackStar, let's set up the numbers themselves:

In [2]:
# the underlying sample with no measurement uncertainty
true_values = np.random.normal(loc = 0, scale = 1, size = 100)

# the sampled true values perturbed my measurement uncertainty for x and y
x = true_values + np.random.normal(loc = 0, scale = 0.1, size = 100)
y = true_values + np.random.normal(loc = 0, scale = 0.1, size = 100)

# half of the z measurements are missing
z = np.array([true_values[i] + np.random.normal(
    loc = 0, scale = 0.3) if i % 2 else float("nan") for i in range(100)])

To construct these arrays, we made use of NumPy's automatic component-wise addition when the arrays are the same length.
Let's inspect the first few elements of each of them:

In [3]:
print("x: ", x[:5])
print("y: ", y[:5])
print("z: ", z[:5])

x:  [-0.2730993   0.11536843 -1.35475868  1.3459729   0.23392896]
y:  [-0.33890861 -0.03828595 -1.4841992   1.3130942   0.27316195]
z:  [        nan -0.3415485          nan  1.09354396         nan]


TrackStar recognizes ``NaN`` values as missing data, so every other value of $z$ is assigned ``NaN`` accordingly (more on this below).

### ``trackstar.sample``: The Workhorse for Storing Data

The workhorse for storing data in TrackStar is ``sample``, which stores data vectors in a dictionary-like manner, using string labels to denote different measured quantities.
The most straight-forward way to construct one is to give it a dictionary containing the arrays of each measured quantity:

In [4]:
data = trackstar.sample({
    "x": x,
    "y": y,
    "z": z,
    "err_x": 100 * [0.1],
    "err_y": 100 * [0.1],
    "err_z": [0.3 if i % 2 else float("nan") for i in range(100)],
})
print(data)

sample(
    N = 100
    x --------------> [-2.7310e-01,  1.1537e-01, -1.3548e+00, ...,  2.4290e+00,  5.8152e-01,  9.5089e-01]
    y --------------> [-3.3891e-01, -3.8286e-02, -1.4842e+00, ...,  2.4613e+00,  6.4115e-01,  9.0541e-01]
    z --------------> [ nan       , -3.4155e-01,  nan       , ...,  2.2425e+00,  nan       ,  6.3844e-01]
)


There are additional ways to load your data into a ``sample`` and additional features that we do not cover in this tutorial.
More examples can be found in out [samples](samples.ipynb) tutorial.

This call returns a ``sample`` object, a data structure with similar behavior to the ``pandas DataFrame`` in that it can be indexed with both row number and column label.
By calling the ``.keys()`` instance method, we are able to access the labels that we have given our data.

In [5]:
print(data.keys())

['x', 'y', 'z']


TrackStar has recognized the measurement uncertainties for what they are based on their labels in our call to ``trackstar.sample`` above (see [below](#where-the-measurement-uncertainties-reside) for information on where they've gone).

We can also access the number of data vectors in our sample by calling either ``len(data)`` or ``data.size``:

In [6]:
print(len(data))
print(data.size)

100
100


By indexing with a row number, we get an individual data vector with all of its components:

In [7]:
print(data[0])
print(data[1])

datum(
    x --------------> -2.7310e-01
    y --------------> -3.3891e-01
)
datum(
    x --------------> 1.1537e-01
    y --------------> -3.8286e-02
    z --------------> -3.4155e-01
)


Note that the ``NaN`` value we entered for our zero'th data vector does not show up here, because TrackStar has recognized this as an indication that there is no measurement.
There are some caveats associated with this behavior, which we recommend new users familiarize themselves with (see the [note on NaNs in TrackStar](#a-note-on-nans-in-trackstar) below).
In short, TrackStar does not allow ``NaN`` values to be changed to numerical values, and vice versa.

---

**NOTE**

While new data vectors can be added to an existing ``sample`` (by calling ``sample.add_datum``), existing data vectors are not allowed to change size.
All vector components must be included upon creation of the data vector.
If a given data vector does not already have a measurement for a particular quantity, then item assignment for that quantity will raise a ``KeyError``.

---

Item assignment can be achieved with familiar standard procedure:

In [8]:
old_value = data["x", 0] # for re-assignment near the end of this cell
data["x", 0] = 0
print(data[0])
data[0, "x"] = 1
print(data[0])
data["x"][0] = 2
print(data[0])
data[0]["x"] = old_value
print(data[0])

datum(
    x --------------> 0.0000e+00
    y --------------> -3.3891e-01
)
datum(
    x --------------> 1.0000e+00
    y --------------> -3.3891e-01
)
datum(
    x --------------> 2.0000e+00
    y --------------> -3.3891e-01
)
datum(
    x --------------> -2.7310e-01
    y --------------> -3.3891e-01
)


The ``sample`` can also be sliced to take different subsamples:

In [9]:
print(data[:5])
print(data[20:25])
print(data[::2])

sample(
    N = 5
    x --------------> [-2.7310e-01,  1.1537e-01, -1.3548e+00,  1.3460e+00,  2.3393e-01]
    y --------------> [-3.3891e-01, -3.8286e-02, -1.4842e+00,  1.3131e+00,  2.7316e-01]
    z --------------> [ nan       , -3.4155e-01,  nan       ,  1.0935e+00,  nan       ]
)
sample(
    N = 5
    x --------------> [ 1.3261e+00, -1.6912e-03, -7.3337e-01,  7.3152e-01, -5.6022e-02]
    y --------------> [ 1.1857e+00, -8.2832e-02, -6.9463e-01,  7.5656e-01, -3.9798e-02]
    z --------------> [ nan       , -1.8502e-02,  nan       ,  1.3787e+00,  nan       ]
)
sample(
    N = 50
    x --------------> [-2.7310e-01, -1.3548e+00,  2.3393e-01, ...,  3.5019e-01, -5.1588e-01,  5.8152e-01]
    y --------------> [-3.3891e-01, -1.4842e+00,  2.7316e-01, ...,  2.2743e-01, -3.1713e-01,  6.4115e-01]
)


By indexing with a column label, we get the measurements of that quantity for each data vector:

In [10]:
print(data["x"])
print(data["z"])

linked_array([-2.7310e-01,  1.1537e-01, -1.3548e+00, ...,  2.4290e+00,  5.8152e-01,  9.5089e-01])
linked_array([ nan       , -3.4155e-01,  nan       , ...,  2.2425e+00,  nan       ,  6.3844e-01])


We can also index the sample with both row number and column label simultaneously:

In [11]:
print(data["x", 0])
print(data[1, "y"])
print(data["z", 0])

-0.2730993026460248
-0.038285954257798564
nan


In general, we recommend indexing with the rule ``data[row, column]`` as opposed to ``data[row][column]`` as it is both faster and more memory efficient.
This recommendation applies to all 2-dimensional data structures in TrackStar (i.e. to the ``track`` object as well).

#### Linked Arrrays

When we indexed our sample with a column label above, we received a particular type of ``array`` as output, namely a ``linked_array``.
This class of array-like objects is special in that its memory is "linked" with our ``sample`` in the sense that modifying its elements *also* modifies the sample.
For example:

In [12]:
x = data["x"]
print(x[0])
old_value = x[0]
x[0] = 0
print(x[0])
print(data["x", 0])
x[0] = old_value
print(data["x", 0])

-0.2730993026460248
0.0
0.0
-0.2730993026460248


By modifying the ``linked_list`` in the above cell, we have also modified the ``"x"`` column of ``data``.
Changes to one variable state also affects the other.

### Where the Measurement Uncertainties Reside

When we called ``trackstar.sample`` above, the dictionary keys labeled ``"err_x"``, ``"err_y"``, and ``"err_z"`` were excluded from the sample and not treated as data vector components.
TrackStar will treat any keys beginning with ``"err_"`` or ending in ``"_err"`` as measurement uncertainties and automatically construct diagonalized covariance matrices for each data vector:

In [13]:
print(data[0].cov)
print(data[1].cov)

covariance matrix(
    [ 1.0000e-02,  0.0000e+00]
    [ 0.0000e+00,  1.0000e-02]
    Quantities: [x, y] (in the order of indexing)
)
covariance matrix(
    [ 1.0000e-02,  0.0000e+00,  0.0000e+00]
    [ 0.0000e+00,  1.0000e-02,  0.0000e+00]
    [ 0.0000e+00,  0.0000e+00,  9.0000e-02]
    Quantities: [x, y, z] (in the order of indexing)
)


If we had not specified any measurement uncertainties in our call to ``trackstar.sample``, each element along the diagonal of the covariance matrix would be given an initial value of $1$.
We could then modify each data vector's covariance matrix by hand after construction of the sample (see example below and further demonstration in [samples](samples.ipynb) tutorial).

The measurement uncertainty on any one quantity can be obtained by taking the square root of the relevant covariance matrix entry along the diagonal.
For example:

In [14]:
print(np.sqrt(data[0].cov["x"]))
print(np.sqrt(data[0].cov["x", "x"]))

0.1
0.1


Note that we received the same result with ``data[0].cov["x"]`` and ``data[0].cov["x", "x"]``.
The covariance matrix is smart enough to know that the user is looking for an element along the diagonal when it is given only one string label or integer index.

If any of the quantities covary, that information can be entered now, after the sample has been constructed.
Both strings and integers are supported; the appropriate integer corresponds to a component-wise match to the string labels returned by the ``.keys()`` function shown above (i.e. ``data[0].cov[0, 1]`` is the same as ``data[0].cov["x", "y"]``).

For example:

In [15]:
data[0].cov["x", "y"] = 0.2
print(data[0].cov)
data[0].cov[0, 1] = 0.3
print(data[0].cov)

covariance matrix(
    [ 1.0000e-02,  2.0000e-01]
    [ 2.0000e-01,  1.0000e-02]
    Quantities: [x, y] (in the order of indexing)
)
covariance matrix(
    [ 1.0000e-02,  3.0000e-01]
    [ 3.0000e-01,  1.0000e-02]
    Quantities: [x, y] (in the order of indexing)
)


Because covariance matrices are symmetric by definition, TrackStar has taken the liberty of modifying not only ``data[0].cov["x", "y"]`` as requested, but ``data[0].cov["y", "x"]`` as well.
Any modification to a covariance matrix changes the components both above and below the diagonal automatically.

We'll simply change the value back to zero, since our mock data do not include any covariance in the input data.

In [16]:
data[0].cov["y", "x"] = 0
print(data[0].cov)

covariance matrix(
    [ 1.0000e-02,  0.0000e+00]
    [ 0.0000e+00,  1.0000e-02]
    Quantities: [x, y] (in the order of indexing)
)


## A Model to Compare with our Mock

Now that we've constructed a ``sample`` (see [above](#a-mock-data-sample)), let's set up our model.
We know that the underlying model is the line $x = y = z$ in 3-dimensional space, so we'll start there.

### ``trackstar.track``: The Workhorse for Storing Models

In the same way that ``trackstar.sample`` does the heavy lifting of data storage, ``trackstar.track`` does the heavy lifting of model storage.
Constructing a ``track`` follows a similar procedure as constructing a ``sample``.
We simply give it a dictionary containing the predicted values at a sufficiently dense sample of points along the predicted curve:

In [17]:
model = trackstar.track({
    "x": np.linspace(-3, 3, 1000),
    "y": np.linspace(-3, 3, 1000),
    "z": np.linspace(-3, 3, 1000)
})
print(model)

track([
       x --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
       y --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
       z --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
       weights --------> [ 1.0000e+00,  1.0000e+00,  1.0000e+00, ...,  1.0000e+00,  1.0000e+00,  1.0000e+00]
])


Note that we gave our ``track`` the **same** column labels as our sample (``"x"``, ``"y"``, and ``"z"``).
When we compute the likelihood functions below, TrackStar will look at the column labels in both the data and the model, and then use the ones that match to determine which vector components should be compared with one another.

Note also that our ``track`` has automatically picked up an additional column ``"weights"``.
The weights describe the *predicted* occurrence rates of data vectors as a function of position (see TrackStar science documentation for further details).
Their values can also be specified in your call to ``trackstar.track`` by either including them as an additional column with the same label or passing them as a keyword argument.

Many of the features available for ``sample`` objects are also available for ``track`` objects.
They also have a ``.keys()`` function, which returns the string labels for different quantities predicted by the model:

In [18]:
print(model.keys())

['x', 'y', 'z']


They also support the ``len`` function, though they store attributes ``n_vectors`` (the number of points the track is sampled on) and ``dim`` (the number of predicted quantities) as opposed to ``sample``'s ``size``:

In [19]:
print(len(model))
print(model.n_vectors)
print(model.dim)

1000
1000
3


---

**NOTE**

Although ``sample`` objects are allowed to grow in size by adding new data vectors, ``track`` objects do not support additional vectors being added.
Every vector at which the model curve is sampled must be supplied to the ``track`` object when it is constructed.
If the ``track`` must change, there are two options: 1) a new track can be created, and 2) a larger track can be constructed with placeholder values (e.g. zeros everywhere) that then get modified when the relevant information is available.

---

``track`` objects can be indexed and sliced with integer indeces to take subsets or coarsened versions of a given track:

In [20]:
print(model[0])
print(model[1])
print(model[:15])
print(model[::2])

linked_dictionary{
  "x": -3.0000e+00,
  "y": -3.0000e+00,
  "z": -3.0000e+00,
  "weights": 1.0000e+00,
}
linked_dictionary{
  "x": -2.9940e+00,
  "y": -2.9940e+00,
  "z": -2.9940e+00,
  "weights": 1.0000e+00,
}
track([
       x --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ..., -2.9279e+00, -2.9219e+00, -2.9159e+00]
       y --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ..., -2.9279e+00, -2.9219e+00, -2.9159e+00]
       z --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ..., -2.9279e+00, -2.9219e+00, -2.9159e+00]
       weights --------> [ 1.0000e+00,  1.0000e+00,  1.0000e+00, ...,  1.0000e+00,  1.0000e+00,  1.0000e+00]
])
track([
       x --------------> [-3.0000e+00, -2.9880e+00, -2.9760e+00, ...,  2.9700e+00,  2.9820e+00,  2.9940e+00]
       y --------------> [-3.0000e+00, -2.9880e+00, -2.9760e+00, ...,  2.9700e+00,  2.9820e+00,  2.9940e+00]
       z --------------> [-3.0000e+00, -2.9880e+00, -2.9760e+00, ...,  2.9700e+00,  2.9820e+00,  2.9940e+00]
      

A ``track`` can also be indexed with strings, or a string and an integer at the same time, just like a ``sample``:

In [21]:
print(model["x"])
print(model["y"])
print(model["x", 0])
print(model["y", -1])

linked_array([-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00])
linked_array([-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00])
-3.0
3.0


Item assignment also follows familiar standard procedures:

In [22]:
old_value = model["x", 0] # for reassignment at the end of this cell
model["x", 0] = 4
print(model[0])
model[0, "x"] = 5
print(model[0])
model["x"][0] = 6
print(model[0])
model[0]["x"] = old_value
print(model[0])

linked_dictionary{
  "x": 4.0000e+00,
  "y": -3.0000e+00,
  "z": -3.0000e+00,
  "weights": 1.0000e+00,
}
linked_dictionary{
  "x": 5.0000e+00,
  "y": -3.0000e+00,
  "z": -3.0000e+00,
  "weights": 1.0000e+00,
}
linked_dictionary{
  "x": 6.0000e+00,
  "y": -3.0000e+00,
  "z": -3.0000e+00,
  "weights": 1.0000e+00,
}
linked_dictionary{
  "x": -3.0000e+00,
  "y": -3.0000e+00,
  "z": -3.0000e+00,
  "weights": 1.0000e+00,
}


#### Linked Dictionaries

When indexing with an integer, as in some of the above cells, we received a ``linked_dictionary`` as output.
The ``linked_dictionary`` is similar to the ``linked_array`` in that it is a ``dict`` whose memory is "linked" with the ``track`` itself.
Modifications to a ``linked_dictionary`` will also be reflected in the ``track``.
For example:

In [23]:
zero = model[0]
zero["x"] = -1
print(zero)
print(model)
zero["x"] = -3
print(zero)
print(model)

linked_dictionary{
  "x": -1.0000e+00,
  "y": -3.0000e+00,
  "z": -3.0000e+00,
  "weights": 1.0000e+00,
}
track([
       x --------------> [-1.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
       y --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
       z --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
       weights --------> [ 1.0000e+00,  1.0000e+00,  1.0000e+00, ...,  1.0000e+00,  1.0000e+00,  1.0000e+00]
])
linked_dictionary{
  "x": -3.0000e+00,
  "y": -3.0000e+00,
  "z": -3.0000e+00,
  "weights": 1.0000e+00,
}
track([
       x --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
       y --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
       z --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
      

In the above cell, modifications to the variable ``zero`` also modified the ``"x"`` column of ``model``.

## Computing the Likelihood Function

At this point, we are ready to compute the likelihood function.
Both individual data vectors and samples have a method ``.loglikelihood()``, which takes in a ``track`` and returns $\ln L$:

In [24]:
print(data[0].loglikelihood(model))
print(data[1].loglikelihood(model))
print(data.loglikelihood(model))

-1.4919181908906873
0.8648797774759773
-604.6523431201857


TrackStar also allows the likelihood function to be computed for only specific quantities.
To do so, simply pass a keyword argument ``quantities`` to the ``loglikelihood`` call, which should be a list of the column labels to use.
For example:

In [25]:
print(data[0].loglikelihood(model, quantities = ["x", "y"]))
print(data.loglikelihood(model, quantities = ["y", "z"]))

-1.4919181908906873
-680.1255641762631


One can also compute the likelihood function for *subsamples* of the data by simply slicing the ``sample``.
For example:

In [26]:
print(data[::2].loglikelihood(model))
print(data[:50].loglikelihood(model))
print(data[-50:].loglikelihood(model))

-302.79339321027766
-303.49316216816766
-301.15918095201823


One can also slice the *model* to compute the likelihood function for only a portion of the predicted ``track``:

In [27]:
print(data.loglikelihood(model[:300]))
print(data.loglikelihood(model[400:600]))
print(data[::2].loglikelihood(model[-200:]))

-1112.885493526313
-739.7014178832077
-1065.4797569010773


### The Importance of the Weights

As mentioned above, the weights quantify the predicted frequency of the data along the curve.
In our initial call above, we were not comparing the data with the *true* input model, because the weights do not reflect the fact that the data follow a Gaussian distribution centered on zero.
To correct this, we assign the weights accordingly:

In [28]:
def gaussian(x, loc = 0, width = 1):
    return np.exp(-(x - loc)**2 / (2 * width)**2)

for i in range(model.n_vectors):
    model["weights", i] = gaussian(model["x", i])

print(model)

track([
       x --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
       y --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
       z --------------> [-3.0000e+00, -2.9940e+00, -2.9880e+00, ...,  2.9880e+00,  2.9940e+00,  3.0000e+00]
       weights --------> [ 1.0540e-01,  1.0635e-01,  1.0731e-01, ...,  1.0731e-01,  1.0635e-01,  1.0540e-01]
])


And now the likelihood that our sample arose from this model has increased significantly:

In [29]:
print(data.loglikelihood(model))

-577.2303901449949


### Is my ``track`` densely sampled enough?

The model prediction, when stored numerically, is treated as a series of discrete line segments as opposed to a smooth curve.
For this reason, it is important to ensure that the ``track`` is sampled at enough points in the observed space.
If not, the fact that the computer sees it as a "jagged" curve is introducing statistically significant numerical artifacts.

One simple way to test this is to simply evaluate the likelihood function for downsampled versions of your ``track`` by slicing it:

In [30]:
# 2, 4, 5, and 10 chosen because they're divisible by the N = 1000 vectors in our track
print(data.loglikelihood(model))
print(data.loglikelihood(model[::2]))
print(data.loglikelihood(model[::4]))
print(data.loglikelihood(model[::5]))
print(data.loglikelihood(model[::10]))

-577.2303901449949
-577.2359990061929
-577.2476227364217
-577.2536459010162
-577.2860466536885


In each of the above cases, the returned value of $\ln L$ changes quite minimally with downsampling, which is an indication that our track is indeed densely sampled enough for the purposes of these calculations.

Another way to check this is to pass the keyword argument ``use_line_segment_corrections = True`` to ``loglikelihood``.
This parameter is ``False`` by default.
When ``True``, TrackStar computes a corrective factor for each individual line segment that accounts for the fact that $\ln L$ is actually changing smoothly from end to end (see TrackStar's science documentation).

In [31]:
print(data.loglikelihood(model))
print(data.loglikelihood(model, use_line_segment_corrections = True))

-577.2303901449949
-577.2412190646513


We see once again that the inferred values of $\ln L$ are quite close to one another.
Between downsampling by slicing and using corrections for the lengths of individual line segments, the former is much less computationally expensive, but the latter computes the *technically correct* value of $\ln L$.

## Parallel Processing

If you have enabled TrackStar's parallel processing features (see directions for installing TrackStar), then making use of these features is easy.
Simply modify the attribute ``n_threads`` of any ``track`` object and proceed as normal:

In [32]:
model.n_threads = 5
print(data.loglikelihood(model))

-577.2303901449948


## A Note on NaNs in TrackStar

Although TrackStar returns ``NaN`` values when a measurement of a particular quantity is not available for a given data vector, these values are not stored in its backend.
**They do not correspond to actual blocks of memory stored on your system.**
As a consequence, the ``NaN`` values that it returns do not correspond to an existing memory address.
For this reason, TrackStar does not allow assignment of ``NaN`` values to real numbers, or vice versa.
Such a change would cause TrackStar to infer an incorrect dimensionality of the vector in question, which could result in memory errors.

As noted in this tutorial, a new data vector or model ``track`` must be created if the dimensionality of some vector is to change.
Alternatively, one can construct a larger vector with its components initialized to some placeholder value (e.g. zero, just not ``NaN``) and then modified thereafter.